In [1]:
{
  "agent_types": {
    "student": {
      "distribution": {
        "count": 50,
        "gender_ratio": {"female": 0.6, "male": 0.4},
        "age_range": [18, 24]
      },
      "attributes": {
        "preferred_temp": {
          "distribution": "uniform",
          "min": 24,
          "max": 26
        },
        "comfort_tolerance": {
          "distribution": "uniform",
          "min": 1.5,
          "max": 2.5
        }
      },
      "behaviors": {
        "device_usage_pattern": "intermittent"
      },
      "schedule_template": "undergraduate_classes"
    },
    "staff": {
      "distribution": {
        "count": 10,
        "gender_ratio": {"female": 0.5, "male": 0.5},
        "age_range": [25, 60]
      },
      "attributes": {
        "preferred_temp": {
          "distribution": "uniform",
          "min": 25,
          "max": 27
        },
        "comfort_tolerance": {
          "distribution": "uniform",
          "min": 1,
          "max": 2
        }
      },
      "behaviors": {
        "device_usage_pattern": "work_hours"
      },
      "schedule_template": "office_hours"
    },
    "cleaner": {
      "distribution": {
        "count": 5,
        "gender_ratio": {"female": 0.4, "male": 0.6},
        "age_range": [20, 55]
      },
      "attributes": {
        "preferred_temp": {
          "distribution": "uniform",
          "min": 26,
          "max": 28
        },
        "comfort_tolerance": {
          "distribution": "uniform",
          "min": 2.5,
          "max": 3.5
        }
      },
      "behaviors": {
        "movement_pattern": "cyclic_route",
        "device_usage_pattern": "low"
      },
      "schedule_template": "cleaning_shifts"
    },
    "warden": {
      "distribution": {
        "count": 2,
        "gender_ratio": {"female": 0.5, "male": 0.5},
        "age_range": [30, 65]
      },
      "attributes": {
        "preferred_temp": {
          "distribution": "uniform",
          "min": 25,
          "max": 27
        },
        "comfort_tolerance": {
          "distribution": "uniform",
          "min": 2,
          "max": 3
        }
      },
      "behaviors": {
        "device_usage_pattern": "monitoring"
      },
      "schedule_template": "warden_duties"
    },
    "visitor": {
      "distribution": {
        "count": 20,
        "gender_ratio": {"female": 0.5, "male": 0.5},
        "age_range": [18, 65]
      },
      "attributes": {
        "preferred_temp": {
          "distribution": "uniform",
          "min": 24,
          "max": 26
        },
        "comfort_tolerance": {
          "distribution": "uniform",
          "min": 1,
          "max": 1.5
        }
      },
      "behaviors": {
        "device_usage_pattern": "short_stay"
      },
      "schedule_template": "visitor_pattern"
    },
    "policy": {
      "distribution": {
        "count": 1
      },
      "attributes": {
        "preferred_temp": {
          "fixed": 25
        },
        "max_delta": 3
      },
      "behaviors": {
        "control_action": ["heat_on", "cool_on", "hold"]
      },
      "schedule_template": "always_on"
    }
  }
}


{'agent_types': {'student': {'distribution': {'count': 50,
    'gender_ratio': {'female': 0.6, 'male': 0.4},
    'age_range': [18, 24]},
   'attributes': {'preferred_temp': {'distribution': 'uniform',
     'min': 24,
     'max': 26},
    'comfort_tolerance': {'distribution': 'uniform', 'min': 1.5, 'max': 2.5}},
   'behaviors': {'device_usage_pattern': 'intermittent'},
   'schedule_template': 'undergraduate_classes'},
  'staff': {'distribution': {'count': 10,
    'gender_ratio': {'female': 0.5, 'male': 0.5},
    'age_range': [25, 60]},
   'attributes': {'preferred_temp': {'distribution': 'uniform',
     'min': 25,
     'max': 27},
    'comfort_tolerance': {'distribution': 'uniform', 'min': 1, 'max': 2}},
   'behaviors': {'device_usage_pattern': 'work_hours'},
   'schedule_template': 'office_hours'},
  'cleaner': {'distribution': {'count': 5,
    'gender_ratio': {'female': 0.4, 'male': 0.6},
    'age_range': [20, 55]},
   'attributes': {'preferred_temp': {'distribution': 'uniform',
     

In [2]:
from mesa import Agent   
import random

class BaseAgent(Agent):
    def __init__(self, unique_id, model, zones, **kwargs):
        super().__init__(unique_id, model)
        self.agent_id = f"{self.__class__.__name__.lower()}_{unique_id}"
        self.agent_type = self.__class__.__name__.replace("Agent", "").lower()
        self.current_room = random.choice(zones) if zones else "Unknown Zone"
        self.preferred_temp = kwargs.get("preferred_temp", 25.0)
        self.comfort_tolerance = kwargs.get("comfort_tolerance", 1.0)
        self.gender = kwargs.get("gender", None)
        self.age = kwargs.get("age", None)
        self.route = kwargs.get("route", [])
        self.max_delta = kwargs.get("max_delta", 3)
        self.using_ac = False
        self.comfort_level = 0.0
        self.current_temp = None
        self.current_day = None
        self.current_hour = None

    def step(self):
        # temp = self.model.get_current_temp(self.current_room) or self.preferred_temp
        # self.current_temp = temp
        temp = self.model.get_current_temp(self.current_room)
        if temp is None:
            temp = float('nan')
        self.current_temp = temp

        self.comfort_level = max(0.0, self.preferred_temp - abs(temp - self.preferred_temp))
        self.using_ac = abs(temp - self.preferred_temp) > self.comfort_tolerance

        self.current_day = getattr(self.model, "current_day", None)
        self.current_hour = getattr(self.model, "current_hour", None)

        self.model.agent_results.append({
            "day": self.current_day,
            "hour": self.current_hour,
            "step": self.model.current_step,
            "agent_id": self.agent_id,
            "agent_type": self.agent_type,
            "room": self.current_room,
            "current_temp": self.current_temp,
            "comfort_level": round(self.comfort_level, 2),
            "using_ac": self.using_ac,
            "preferred_temp": self.preferred_temp
        })

class StudentAgent(BaseAgent): pass
class StaffAgent(BaseAgent): pass
class CleanerAgent(BaseAgent): pass
class WardenAgent(BaseAgent): pass
class VisitorAgent(BaseAgent): pass
class PolicyAgent(BaseAgent): pass

In [3]:
# utils.py
import random
import json

def sample_value(attr_cfg):
    if attr_cfg is None:
        return None

    dist = attr_cfg.get("distribution")
    if dist == "uniform":
        return random.uniform(attr_cfg["min"], attr_cfg["max"])
    if dist == "normal":
        return random.gauss(attr_cfg["mean"], attr_cfg["std_dev"])
    if "fixed" in attr_cfg:
        return attr_cfg["fixed"]
    if "value" in attr_cfg:
        return attr_cfg["value"]
    if "min" in attr_cfg and "max" in attr_cfg:
        return random.uniform(attr_cfg["min"], attr_cfg["max"])
    return None

def sample_gender(gender_ratio):
    """gender_ratio: {'female':0.6,'male':0.4}"""
    if not gender_ratio:
        return None
    choices = list(gender_ratio.keys())
    weights = list(gender_ratio.values())
    return random.choices(choices, weights=weights, k=1)[0]

def sample_age(age_range):
    if not age_range:
        return None
    return random.randint(age_range[0], age_range[1])

def sample_agent_attributes(agent_type, agents_file="agents.json"):
    with open(agents_file, "r") as f:
        data = json.load(f)

    agent_info = data["agent_types"].get(agent_type)
    if not agent_info:
        raise ValueError(f"Agent type '{agent_type}' not found in {agents_file}")

    attrs = {}

    # attributes
    for attr_name, cfg in agent_info.get("attributes", {}).items():
        attrs[attr_name] = sample_value(cfg)

    dist_cfg = agent_info.get("distribution", {})
    attrs["gender"] = sample_gender(dist_cfg.get("gender_ratio"))
    attrs["age"] = sample_age(dist_cfg.get("age_range"))

    return attrs


In [4]:
import sys
sys.path.append("/project/lt200291-ignite/Project_chomwong/energyplus/EnergyPlus-25.1.0-1c11a3d85f-Linux-CentOS7.9.2009-x86_64")  # ตัวอย่าง path, ตรวจสอบจริงด้วย `module show energyplus`
from pyenergyplus.api import EnergyPlusAPI

import os
import json
import random
import pandas as pd
from mesa import Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

from agent import StudentAgent, StaffAgent, CleanerAgent, WardenAgent, VisitorAgent, PolicyAgent
from utils import sample_value, sample_gender, sample_age

from eppy.modeleditor import IDF


class BuildingModel(Model):
    def __init__(self, config, agents_file=None, idf_path=None, ep_control=False):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.config = config
        self.current_step = 0
        self.ep_control = ep_control
        self.zones = config["mesa"].get("zones", [])
        self.last_zone_temps = {z: None for z in self.zones}
        self.agent_results = []

        self.current_day = None
        self.current_hour = None

        # ----------------- DataCollector -----------------
        self.datacollector = DataCollector(
            model_reporters={
                f"AvgTemp_{zone.replace(' ', '_')}": (lambda m, z=zone: m.get_current_temp(z))
                for zone in self.zones
            }
        )

        # ----------------- JSON (agents) -----------------
        if agents_file and os.path.exists(agents_file):
            with open(agents_file, "r") as f:
                self.agent_config = json.load(f)
            self._create_agents_from_json()
        else:
            print("[WARN] agents_file ไม่พบหรือไม่ระบุ → จะไม่มี agent ถูกสร้าง")

        # ----------------- EnergyPlus Setup -----------------
        if self.ep_control:
            idf_zone_names = []
            if idf_path:
                try:
                    idd_path = "/project/lt200291-ignite/Project_chomwong/energyplus/EnergyPlus-25.1.0-1c11a3d85f-Linux-CentOS7.9.2009-x86_64/Energy+.idd"
                    IDF.setiddname(idd_path)
                    idf = IDF(idf_path)
                    idf_zone_names = [z.Name for z in idf.idfobjects.get('ZONE', [])]
                except Exception as e:
                    print(f"[WARN] ไม่สามารถโหลด IDF: {e}")
                    idf_zone_names = []

            self.api = EnergyPlusAPI()
            self.state = self.api.state_manager.new_state()
            self.exchange = self.api.exchange

            def normalize(name):
                return name.lower().strip().replace(" ", "_")

            norm_idf_zones = {normalize(z): z for z in idf_zone_names}
            self.zone_name_map = {z: norm_idf_zones.get(normalize(z), None) for z in self.zones}
            print("Zone map:", self.zone_name_map)

            self.zone_temp_handles = {}
            self.zone_setpoint_handles = {}

            def setup_handles_first_timestep(state):
                for zone in self.zones:
                    ep_zone = self.zone_name_map.get(zone)
                    if ep_zone is None:
                        print(f"[WARN] ไม่มี mapping ระหว่าง zone '{zone}' กับ IDF zone names")
                        continue

                    temp_handle = self.exchange.get_variable_handle(state, "Zone Air Temperature", ep_zone)
                    if temp_handle == -1:
                        print(f"[WARN] ไม่พบ variable 'Zone Air Temperature' ของ zone '{zone}' (ep zone: {ep_zone})")

                    sp_handle = self.exchange.get_actuator_handle(state, "Zone Temperature Control", "SecondarySchool ClgSetp", ep_zone)
                    if sp_handle == -1:
                        print(f"[WARN] ไม่พบ actuator 'Cooling Setpoints' ของ zone '{zone}' (ep zone: {ep_zone})")

                    self.zone_temp_handles[zone] = temp_handle
                    self.zone_setpoint_handles[zone] = sp_handle

                print("Zone temp handles:", self.zone_temp_handles)
                print("Zone setpoint handles:", self.zone_setpoint_handles)

            self.api.runtime.callback_after_predictor_after_hvac_managers(
                self.state, setup_handles_first_timestep
            )

    # ============================================================
    # agents creation
    # ============================================================
    def _create_agents_from_json(self):
        mapping = {
            "student": StudentAgent,
            "staff": StaffAgent,
            "cleaner": CleanerAgent,
            "warden": WardenAgent,
            "visitor": VisitorAgent,
            "policy": PolicyAgent
        }

        total_created = 0
        for agent_type, info in self.agent_config.get("agent_types", {}).items():
            dist = info.get("distribution", {})
            count = dist.get("count", 1)

            for i in range(count):
                preferred_temp_attr = info.get("attributes", {}).get("preferred_temp", {"distribution": "uniform", "min": 25, "max": 25})
                comfort_tolerance_attr = info.get("attributes", {}).get("comfort_tolerance", {"distribution": "uniform", "min": 1, "max": 1})

                preferred_temp = sample_value(preferred_temp_attr)
                comfort_tolerance = sample_value(comfort_tolerance_attr)

                room = random.choice(self.zones) if self.zones else "Unknown Zone"

                AgentClass = mapping.get(agent_type, StudentAgent)
                agent = AgentClass(
                    unique_id=f"{agent_type}_{i}",
                    model=self,
                    zones=self.zones,
                    current_room=room,
                    preferred_temp=preferred_temp,
                    comfort_tolerance=comfort_tolerance,
                    agent_type=agent_type
                )
                self.schedule.add(agent)
                total_created += 1

        print(f"✅ โหลด agent สำเร็จทั้งหมด: {total_created} ตัว")


    # ============================================================
    #  EP-control methods
    # ============================================================
    def get_current_temp(self, zone_name):
        return self.last_zone_temps.get(zone_name)

    def compute_setpoint_requests(self):
        requests = {}
        per_zone = {}
        for agent in self.schedule.agents:
            if getattr(agent, "using_ac", False) and agent.current_room:
                per_zone.setdefault(agent.current_room, []).append(agent.preferred_temp)
        for zone, temps in per_zone.items():
            temps = [t for t in temps if t is not None]
            if temps:
                requests[zone] = min(temps)
        return requests

    def read_zone_temps_from_ep(self):
        temps = {}
        for zone in self.zones:
            handle = self.zone_temp_handles.get(zone, -1)
            if handle in [None, -1]:
                temps[zone] = None
            else:
                try:
                    temps[zone] = self.exchange.get_variable_value(self.state, handle)
                except Exception:
                    temps[zone] = None
        return temps

    def apply_setpoints_to_ep(self, setpoint_map: dict):
        for zone, val in (setpoint_map or {}).items():
            handle = self.zone_setpoint_handles.get(zone, -1)
            if handle in [None, -1]:
                continue
            try:
                self.exchange.set_actuator_value(self.state, handle, val)
            except Exception:
                continue

    # ============================================================
    #  Agent step logic
    # ============================================================
    def step_agents(self, ep_model=None):
        if ep_model is not None:
            zone_temps = ep_model.read_zone_temps_from_ep()
            for z, t in zone_temps.items():
                if t is not None:
                    self.last_zone_temps[z] = t

        for agent in self.schedule.agents:
            temp = self.last_zone_temps.get(agent.current_room)
            agent.current_temp = temp

            if temp is not None:
                try:
                    agent.comfort_level = max(0.0, agent.preferred_temp - abs(temp - agent.preferred_temp))
                except Exception:
                    agent.comfort_level = None

                agent.using_ac = abs(temp - agent.preferred_temp) > agent.comfort_tolerance
            else:
                agent.comfort_level = None
                agent.using_ac = False

            agent.current_day = getattr(self, "current_day", None)
            agent.current_hour = getattr(self, "current_hour", None)

            self.agent_results.append({
                "day": agent.current_day,
                "hour": agent.current_hour,
                "step": self.current_step,
                "agent_id": agent.unique_id,
                "agent_type": agent.agent_type,
                "room": agent.current_room,
                "current_temp": agent.current_temp,
                "comfort_level": round(agent.comfort_level, 2) if agent.comfort_level is not None else None,
                "using_ac": agent.using_ac,
                "preferred_temp": agent.preferred_temp
            })

        try:
            self.datacollector.collect(self)
        except Exception:
            pass

        self.current_step += 1

    def collect_agent_results(self):
        return getattr(self, "agent_results", [])

    # ============================================================
    #  Export results
    # ============================================================
    def export_zone_csv(self, filename="zone_results.csv"):
        df = self.datacollector.get_model_vars_dataframe()
        df.to_csv(filename)
        print(f"Zone-level results saved to {filename}")

    def export_agent_csv(self, filename="agent_results.csv"):
        df = pd.DataFrame(self.agent_results)
        df.to_csv(filename, index=False)
        print(f"Agent-level results saved to {filename}")

    # # ============================================================
    # #  Quick run + export
    # # ============================================================
    # def run_simulation_and_export(self, steps, zone_file="zone_results.csv", agent_file="agent_results.csv"):
    #     for _ in range(steps):
    #         self.step_agents(ep_model=self if self.ep_control else None)
    #     self.export_zone_csv(zone_file)
    #     self.export_agent_csv(agent_file)


In [ ]:
import os
os.environ["ENERGYPLUS_EXE"] = "/project/lt200291-ignite/Project_chomwong/energyplus/EnergyPlus-25.1.0-1c11a3d85f-Linux-CentOS7.9.2009-2023-x86_64/energyplus"
os.environ["NCCL_DEBUG"] = "INFO"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["TORCH_NCCL_TIMEOUT"] = "1200"
os.environ["TORCH_DISTRIBUTED_DEBUG"] = "DETAIL"

import yaml
import json
import pandas as pd
import torch
import torch.distributed as dist
import queue
import threading
from model import BuildingModel

def init_distributed():
    rank = int(os.environ.get("RANK", 0))
    world_size = int(os.environ.get("WORLD_SIZE", 1))
    gpu_count = torch.cuda.device_count()

    if world_size > 1:
        backend = "nccl" if torch.cuda.is_available() and rank < gpu_count else "gloo"
        dist.init_process_group(
            backend=backend,
            rank=rank,
            world_size=world_size,
            init_method="env://"
        )
    else:
        backend = "gloo"

    device = torch.device(f"cuda:{rank}") if backend == "nccl" else torch.device("cpu")
    if backend == "nccl":
        torch.cuda.set_device(device)
    print(f"[Rank {rank}] Using backend={backend}, device={device}", flush=True)
    return rank, world_size, device, backend

def dict_to_tensor(d, keys, device):
    return torch.tensor([d.get(k, float('inf')) for k in keys], dtype=torch.float32, device=device)

def tensor_to_dict(tensor, keys):
    return {k: float(v) for k, v in zip(keys, tensor.tolist())}

def run_simulation():
    with open("config.yaml", "r") as f:
        config = yaml.safe_load(f)
    with open("agents.json", "r") as f:
        agents_json = json.load(f)

    rank, world_size, device, backend = init_distributed()
    steps = config["mesa"]["steps"]
    zone_keys = config["mesa"]["zones"]
    print("Zones:", zone_keys, flush=True)

    callback_queue = queue.Queue()

    model_agents = BuildingModel(config, "agents.json", ep_control=False)

    ep_model = None
    if rank == 0:
        idf_path = "/project/lt200291-ignite/Project_chomwong/project/EnergyPlus_BP_Boonchoo/output/expanded.idf"
        weather_path = "/project/lt200291-ignite/Project_chomwong/project/EnergyPlus_BP_Boonchoo/output/in.epw"
        ep_model = BuildingModel(config, "agents.json", ep_control=True, idf_path=idf_path) #!#
        os.makedirs("outEnergyPlusBoonchoo", exist_ok=True)

        def ep_agent_callback(state):
            print("[Callback] Triggered", flush=True)

            zone_temps = ep_model.read_zone_temps_from_ep()
            print("Current zone temperatures in EP:", flush=True)
            for zone, temp in zone_temps.items():
                print(f"  {zone}: {temp if temp is not None else 'N/A'}", flush=True)
            
            ep_model.last_zone_temps.update(zone_temps)
            
            callback_queue.put(zone_temps)
            
            local_requests = ep_model.compute_setpoint_requests()
            ep_model.apply_setpoints_to_ep(local_requests)

        ep_model.api.runtime.callback_after_predictor_after_hvac_managers(
            ep_model.state, ep_agent_callback
        )

        ep_args = ["-d", "./outEnergyPlusBoonchoo", "-w", weather_path, idf_path]
        ep_thread = threading.Thread(target=lambda: ep_model.api.runtime.run_energyplus(ep_model.state, ep_args))
        ep_thread.start()
    else:
        ep_thread = None

    for step in range(steps):
        if rank == 0:
            zone_temps = callback_queue.get()
            #model_agents.last_zone_temps.update(zone_temps) #!#
            ep_model.last_zone_temps.update(zone_temps)
    
            #model_agents.step_agents(ep_model=None) #!#
            ep_model.step_agents(ep_model=ep_model)
    
            for agent in model_agents.schedule.agents:
                print(f"{agent.unique_id}: current_temp={agent.current_temp}, wants={agent.preferred_temp}, using_ac={agent.using_ac}", flush=True)
    
        local_requests = model_agents.compute_setpoint_requests() if rank == 0 else {k: float('inf') for k in zone_keys}
        local_tensor = dict_to_tensor(local_requests, zone_keys, device)
    
        if world_size > 1:
            gathered = [torch.zeros_like(local_tensor) for _ in range(world_size)]
            dist.all_gather(gathered, local_tensor)
            merged_tensor = torch.min(torch.stack(gathered), dim=0).values
        else:
            merged_tensor = local_tensor
    
        merged_dict = tensor_to_dict(merged_tensor, zone_keys)
    
        if rank == 0 and ep_model:
            ep_model.apply_setpoints_to_ep(merged_dict)
            print("Applied setpoints to EP:", flush=True)
            for z, val in merged_dict.items():
                print(f"  {z}: {val:.2f} °C", flush=True)
    
        if (step + 1) % max(1, steps // 5) == 0:
            print(f"[Rank {rank}] Step {step+1}/{steps}", flush=True)
    
        if world_size > 1:
            dist.barrier()

    if rank == 0 and ep_thread:
        ep_thread.join()

    local_results = model_agents.collect_agent_results()
    if world_size > 1:
        dist.barrier()
        gathered_results = [None] * world_size
        dist.all_gather_object(gathered_results, local_results)
        if rank == 0:
            combined = []
            for sub in gathered_results:
                if sub:
                    combined.extend(sub)
            df = pd.DataFrame(combined)
            df.to_csv("mesa_agent_results.csv", index=False)
            print("[Rank 0] Results saved.", flush=True)
        dist.destroy_process_group()
    else:
        df = pd.DataFrame(local_results)
        df.to_csv("mesa_agent_results.csv", index=False)
        print("[Single rank] Results saved.", flush=True)
    if rank == 0:
        try:
            os.makedirs("mesa_out_result", exist_ok=True)
            ep_model.export_agent_csv("mesa_out_result/mesa_agent_results.csv")
            ep_model.export_zone_csv("mesa_out_result/mesa_zone_results.csv")
            print("[Rank 0] Exported EnergyPlus + Agent results to mesa_out_result/", flush=True)
        except Exception as e:
            print(f"[ERROR] Export failed: {e}", flush=True)

if __name__ == "__main__":
    run_simulation()


Couldn't import dot_parser, loading of dot files will not be possible.
[Rank 0] Using backend=gloo, device=cpu
Zones: ['Zone_Classroom_7302', 'Zone_Classroom_7315']
✅ โหลด agent สำเร็จทั้งหมด: 88 ตัว
✅ โหลด agent สำเร็จทั้งหมด: 88 ตัว
Zone map: {'Zone_Classroom_7302': 'Zone_Classroom_7302', 'Zone_Classroom_7315': 'Zone_Classroom_7315'}
EnergyPlus Starting
EnergyPlus, Version 25.1.0-1c11a3d85f, YMD=2025.10.19 14:49
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
Current zone temperatures in EP:
  Zone_Classroom_7302: 23.0
  Zone_Classroom_7315: 23.0
student_0: current_temp=None, wants=24.561601146855196, using_ac=False
student_1: current_temp=None, wants=25.751199

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



staff_7: current_temp=None, wants=25.435638260013484, using_ac=False
staff_8: current_temp=None, wants=25.757152124093786, using_ac=False
staff_9: current_temp=None, wants=26.96495140104565, using_ac=False
cleaner_0: current_temp=None, wants=26.926201053201694, using_ac=False
cleaner_1: current_temp=None, wants=27.999566976407415, using_ac=False
cleaner_2: current_temp=None, wants=27.70208499234543, using_ac=False
cleaner_3: current_temp=None, wants=27.098481472528825, using_ac=False
cleaner_4: current_temp=None, wants=26.452086239635065, using_ac=False
warden_0: current_temp=None, wants=25.664355730480604, using_ac=False
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
warden_1: current_temp=None, wants=26.99468674033558, using_ac=False
Current zone temperatures in EP:
visitor_0: current_temp=None, wants=24.033077573939643, using_ac=False
  Zone_Classroom_7

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



student_16: current_temp=None, wants=24.962549544942433, using_ac=False
student_17: current_temp=None, wants=24.22498480199744, using_ac=False
student_18: current_temp=None, wants=25.94911998564007, using_ac=False
student_19: current_temp=None, wants=24.907002640997487, using_ac=False
student_20: current_temp=None, wants=25.053586087386318, using_ac=False
student_21: current_temp=None, wants=24.822896298832486, using_ac=False
student_22: current_temp=None, wants=24.736347404870934, using_ac=False
student_23: current_temp=None, wants=24.858980352823274, using_ac=False
student_24: current_temp=None, wants=24.52836358643336, using_ac=False
student_25: current_temp=None, wants=24.269275191166344, using_ac=False
student_26: current_temp=None, wants=25.82634720502253, using_ac=False
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
student_27: current_temp=None, wa

...

Applied setpoints to EP:
  Zone_Classroom_7302: inf °C
  Zone_Classroom_7315: inf °C
student_0: current_temp=None, wants=24.561601146855196, using_ac=False
student_1: current_temp=None, wants=25.751199046621565, using_ac=False
student_2: current_temp=None, wants=24.20124874262165, using_ac=False
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
Current zone temperatures in EP:
student_3: current_temp=None, wants=25.180939298166084, using_ac=False
student_4: current_temp=None, wants=25.400705347971062, using_ac=False
  Zone_Classroom_7302: 24.852013779583686
  Zone_Classroom_7315: 26.294517142916437
student_5: current_temp=None, wants=25.2786302590696, using_ac=False
student_6: current_temp=None, wants=24.45847610259585, using_ac=False
student_7: current_temp=None, wants=24.66682892822065, using_ac=False
student_8: current_temp=None, wants=24.60628879333415, u

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Current zone temperatures in EP:
  Zone_Classroom_7302: 24.053317424536107
  Zone_Classroom_7315: 24.09343880710805
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
Current zone temperatures in EP:
  Zone_Classroom_7302: 24.053317424536104
  Zone_Classroom_7315: 24.093438807108054
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
Current zone temperatures in EP:
  Zone_Classroom_7302: 24.05331742453611
  Zone_Classroom_7315: 24.093438807108047
Zone temp handles: {'Zone_Classroom_7302': 650, 'Zone_Classroom_7315': 660}
Zone setpoint handles: {'Zone_Classroom_7302': 8857, 'Zone_Classroom_7315': 8861}
[Callback] Triggered
Current zone temperatures in EP:
  Zone_Classroom_7302: 24.053317424536107
  Zone_Classroom_7315: 24.0934388071080

EnergyPlus Completed Successfully.


Agent-level results saved to mesa_out_result/mesa_agent_results.csv
Zone-level results saved to mesa_out_result/mesa_zone_results.csv
[Rank 0] Exported EnergyPlus + Agent results to mesa_out_result/
